**Converting Labels to Detection**

In [ ]:
!pip install ultralytics opencv-python torch torchvision torchaudio psutil pyqt5

from google.colab import drive
drive.mount('/content/drive')
import os
import glob
import numpy as np
import time

LABELS_DIR = '/content/drive/MyDrive/Regnum/Labels'
OUTPUT_LABELS_DIR = '/content/drive/MyDrive/Regnum/Labels_detection'

os.makedirs(OUTPUT_LABELS_DIR, exist_ok=True)

def polygon_to_bbox(coords):
    coords = np.array(coords).reshape(-1, 2)
    x_min, y_min = coords.min(axis=0)
    x_max, y_max = coords.max(axis=0)
    cx = (x_min + x_max) / 2
    cy = (y_min + y_max) / 2
    w = x_max - x_min
    h = y_max - y_min
    return [cx, cy, w, h]

for label_path in glob.glob(os.path.join(LABELS_DIR, '*.txt')):
    base_name = os.path.basename(label_path)
    output_path = os.path.join(OUTPUT_LABELS_DIR, base_name)

    with open(label_path, 'r') as f:
        lines = f.readlines()

    new_lines = []
    for line in lines:
        parts = line.strip().split()
        if not parts:
            continue
        class_id = int(parts[0])
        coords = [float(p) for p in parts[1:]]

        if len(coords) == 4:
            bbox = coords
        elif len(coords) % 2 == 0 and len(coords) >= 6:

            bbox = polygon_to_bbox(coords)
        else:
            print(f"Skipping invalid line in {base_name}: {line.strip()}")
            continue

        new_line = f"{class_id} {' '.join(f'{v:.6f}' for v in bbox)}\n"
        new_lines.append(new_line)

    if new_lines:
        with open(output_path, 'w') as f:
            f.writelines(new_lines)
    else:
        print(f"No valid annotations in {base_name} → skipping")

print("Conversion complete")
print(f"Converted labels: {len(os.listdir(OUTPUT_LABELS_DIR))}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 118.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.4/282.4 kB 24.2 MB/s eta 0:00:00
Mounted at /content/drive
Conversion complete
Converted labels: 456


Install dependencies & mount drive

In [ ]:
!pip install -q ultralytics opencv-python psutil pyqt5 torch torchvision torchaudio pynvml

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Train / Validation Split**

In [ ]:
import os
import shutil
import random
from pathlib import Path

BASE_DIR = Path('/content/drive/MyDrive/Regnum')
IMAGES_DIR = BASE_DIR / 'Images'
LABELS_DIR = BASE_DIR / 'Labels_detection'

NEW_BASE = BASE_DIR / 'dataset_split'
TRAIN_IMG = NEW_BASE / 'images' / 'train'
TRAIN_LBL = NEW_BASE / 'labels' / 'train'
VAL_IMG   = NEW_BASE / 'images' / 'val'
VAL_LBL   = NEW_BASE / 'labels' / 'val'

for d in [TRAIN_IMG, TRAIN_LBL, VAL_IMG, VAL_LBL]:
    d.mkdir(parents=True, exist_ok=True)

image_files = list(IMAGES_DIR.glob('**/*.[jJ][pP][gG]')) + list(IMAGES_DIR.glob('**/*.[pP][nN][gG]'))
print(f"Found {len(image_files)} images")

if len(image_files) == 0:
    raise ValueError("No images found! Check path and extensions.")

print("Sample images:", [str(f) for f in image_files[:5]])

random.seed(42)
random.shuffle(image_files)
split_idx = int(0.8 * len(image_files))
train_files = image_files[:split_idx]
val_files   = image_files[split_idx:]

print(f"Train: {len(train_files)} images | Val: {len(val_files)} images")

def copy_pair(img_path, dest_img_dir, dest_lbl_dir):
    lbl_name = img_path.with_suffix('.txt').name
    lbl_path = LABELS_DIR / lbl_name
    if lbl_path.exists():
        shutil.copy(img_path, dest_img_dir / img_path.name)
        shutil.copy(lbl_path, dest_lbl_dir / lbl_name)
    else:
        print(f"Warning: No label for {img_path.name} → skipping image copy")
        return False
    return True

train_copied = 0
for img in train_files:
    if copy_pair(img, TRAIN_IMG, TRAIN_LBL):
        train_copied += 1

val_copied = 0
for img in val_files:
    if copy_pair(img, VAL_IMG, VAL_LBL):
        val_copied += 1

print(f"Train copied: {train_copied} | Val copied: {val_copied}")
print("Dataset split complete.")

Found 456 images
Sample images: ['/content/drive/MyDrive/Regnum/Images/100_jpg.rf.49e80e5a51a7fac3fe774aa4a879939c.jpg', '/content/drive/MyDrive/Regnum/Images/103_jpg.rf.3229a6787fa013854a7e75d19969f258.jpg', '/content/drive/MyDrive/Regnum/Images/109_jpg.rf.a056ed245a8f7716fae3ce4d2e0c7137.jpg', '/content/drive/MyDrive/Regnum/Images/111_jpg.rf.414ed79e9747b566a22d87aa07f2f28b.jpg', '/content/drive/MyDrive/Regnum/Images/102_jpg.rf.a21da2fc12e83c9531ee46e5c9829f30.jpg']
Train: 364 images | Val: 92 images
Train copied: 364 | Val copied: 92
Dataset split complete.


Creating data.yaml

In [ ]:
CLASS_NAMES = [
    'Auto Rickshaw', 'Bus', 'CNG', 'Cycle Rickshaw', 'Jeep - SUV', 'Microbus', 'Motorcycle', 'Private Sedan Car', 'Tempo', 'Trailer', 'Truck'
]

yaml_content = f"""\
path: {BASE_DIR / 'dataset_split'}
train: images/train
val: images/val

nc: {len(CLASS_NAMES)}
names: {CLASS_NAMES}
"""

yaml_path = BASE_DIR / 'data.yaml'
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print(f"data.yaml created at: {yaml_path}")

data.yaml created at: /content/drive/MyDrive/Regnum/data.yaml


Training **YOLOv8m**

In [ ]:
from ultralytics import YOLO
from pathlib import Path
import torch
import os

print("Checking GPU availability...")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available? {torch.cuda.is_available()}")
print(f"CUDA device count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    device = 'cuda:0'
    print("GPU detected! Using Tesla T4 (or similar).")
    !nvidia-smi
else:
    device = 'cpu'
    print("WARNING: No GPU found! Falling back to CPU — training will be VERY SLOW. Enable GPU via Runtime → Change runtime type → T4 GPU → Save → Restart runtime.")


BASE_DIR = Path('/content/drive/MyDrive/Regnum')
DATA_YAML = BASE_DIR / 'data.yaml'

MODEL_SAVE_DIR = BASE_DIR / 'runs' / 'regnum_traffic_model'
LAST_WEIGHTS = MODEL_SAVE_DIR / 'weights' / 'last.pt'


if LAST_WEIGHTS.exists():
    print("Found previous checkpoint → Resuming training from last.pt")
    model = YOLO(str(LAST_WEIGHTS))
    resume = True
else:
    print("No checkpoint found → Starting fresh training with yolov8m.pt")
    model = YOLO('yolov8m.pt')
    resume = False


print(f"Using data.yaml: {DATA_YAML}")
print(f"Expected train images: {BASE_DIR / 'dataset_split' / 'images' / 'train'}")
print(f"Expected val images:   {BASE_DIR / 'dataset_split' / 'images' / 'val'}")


results = model.train(
    data=str(DATA_YAML),
    epochs=50,
    imgsz=640,
    batch=16,
    device=device,
    workers=2,
    patience=15,
    save_period=1,
    project=str(BASE_DIR / 'runs'),
    name='regnum_traffic_model',
    exist_ok=True,
    augment=True,
)

print("\n" + "="*60)
print("Training finished or stopped early.")
print(f"Best weights:   {MODEL_SAVE_DIR}/weights/best.pt")
print(f"Last checkpoint: {MODEL_SAVE_DIR}/weights/last.pt")
print(f"Logs & plots:    {MODEL_SAVE_DIR}")
print("="*60)
print(f"\nUse this for inference/GUI: {MODEL_SAVE_DIR}/weights/best.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Checking GPU availability...
PyTorch version: 2.9.0+cu126
CUDA available? True
CUDA device count: 1
GPU detected! Using Tesla T4 (or similar).
Thu Jan 29 07:48:32 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |           

Export to **ONNX** & Validate Model

In [ ]:
best_model_path = MODEL_SAVE_DIR / 'weights' / 'best.pt'
model = YOLO(str(best_model_path))
model.export(format='onnx', dynamic=True, simplify=True)

print(f"ONNX model saved: {MODEL_SAVE_DIR}/weights/best.onnx")

metrics = model.val()
print("\nValidation metrics:")
print(f"mAP@0.5     : {metrics.box.map:.4f}")
print(f"mAP@0.5:0.95: {metrics.box.map50:.4f}")
print(f"Precision   : {metrics.box.p.mean():.4f}")
print(f"Recall      : {metrics.box.r.mean():.4f}")

Ultralytics 8.4.8 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
Model summary (fused): 93 layers, 25,846,129 parameters, 0 gradients, 78.7 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/Regnum/runs/regnum_traffic_model/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 15, 8400) (49.6 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<2.0.0', 'onnxslim>=0.1.71', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 14 packages in 209ms
Prepared 6 packages in 6.02s
Installed 6 packages in 260ms
 + colorama==0.4.6
 + coloredlogs==15.0.1
 + humanfriendly==10.0
 + onnx==1.20.1
 + onnxruntime-gpu==1.23.2
 + onnxslim==0.1.82

requirements: AutoUpdate success ✅ 7.6s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to 

/usr/local/lib/python3.12/dist-packages/torch/onnx/_internal/torchscript_exporter/utils.py:1447: OnnxExporterWarning: Exporting to ONNX opset version 22 is not supported. by 'torch.onnx.export()'. The highest opset version supported is 20. To use a newer opset version, consider 'torch.onnx.export(..., dynamo=True)'. 
  warnings.warn(


ONNX: slimming with onnxslim 0.1.82...
ONNX: export success ✅ 20.1s, saved as '/content/drive/MyDrive/Regnum/runs/regnum_traffic_model/weights/best.onnx' (99.3 MB)

Export complete (22.9s)
Results saved to /content/drive/MyDrive/Regnum/runs/regnum_traffic_model/weights
Predict:         yolo predict task=detect model=/content/drive/MyDrive/Regnum/runs/regnum_traffic_model/weights/best.onnx imgsz=640 
Validate:        yolo val task=detect model=/content/drive/MyDrive/Regnum/runs/regnum_traffic_model/weights/best.onnx imgsz=640 data=/content/drive/MyDrive/Regnum/data.yaml  
Visualize:       https://netron.app
ONNX model saved: /content/drive/MyDrive/Regnum/runs/regnum_traffic_model/weights/best.onnx
Ultralytics 8.4.8 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 93 layers, 25,846,129 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.4±0.2 ms, read: 31.2±6.4 MB/s, size: 50.7 KB)
val: Scanning /content/drive/MyDrive/Regnum/dataset

Full Video Processing

In [ ]:
from ultralytics import YOLO
import cv2
import time
import numpy as np
from collections import deque
from pathlib import Path

BASE_DIR = Path('/content/drive/MyDrive/Regnum')
VIDEO_PATH = BASE_DIR / 'Video' / 'Supporting video for Dataset-2.mp4'
FULL_OUTPUT = BASE_DIR / 'full_processed_Dataset2.mp4'
MODEL_PATH = BASE_DIR / 'runs' / 'regnum_traffic_model' / 'weights' / 'best.pt'

for p, name in [(VIDEO_PATH, "Video"), (MODEL_PATH, "Model")]:
    if not p.exists():
        print(f"Error: {name} not found → {p}")
        raise FileNotFoundError(f"{name} missing")

print("Loading model...")
model = YOLO(str(MODEL_PATH))
print("Model loaded OK")

cap = cv2.VideoCapture(str(VIDEO_PATH))
if not cap.isOpened():
    raise RuntimeError("Cannot open video")

fps_src = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Video: {w}x{h} @ {fps_src:.1f} FPS | {total_frames} frames (~{total_frames/fps_src:.0f} sec)")

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(str(FULL_OUTPUT), fourcc, fps_src, (w, h))

frame_count = 0
times = []
fps_history = deque(maxlen=30)
start_total = time.time()

print("Starting full inference...")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video reached.")
        break

    t0 = time.time()
    results = model.track(frame, persist=True, tracker="botsort.yaml", verbose=False)
    t1 = time.time()

    process_time = t1 - t0
    times.append(process_time)
    current_fps = 1 / process_time if process_time > 0 else 0
    fps_history.append(current_fps)

    annotated = results[0].plot() if results[0].boxes is not None else frame.copy()
    out.write(annotated)

    frame_count += 1

    if frame_count % 50 == 0:
        avg_so_far = sum(fps_history) / len(fps_history) if fps_history else 0
        elapsed = time.time() - start_total
        eta_min = ((total_frames - frame_count) * (elapsed / frame_count)) / 60 if frame_count > 0 else 0
        print(f"Frame {frame_count:6d}/{total_frames} | "
              f"Inst FPS: {current_fps:.1f} | Avg: {avg_so_far:.1f} | ETA ~{eta_min:.1f} min")

cap.release()
out.release()

total_time = time.time() - start_total
avg_fps = len(times) / sum(times) if times else 0

print("\n" + "="*70)
print(f"Finished – processed {frame_count} / {total_frames} frames")
print(f"Total wall time: {total_time:.1f} s")
print(f"Average FPS: {avg_fps:.1f}")
print(f"Saved to: {FULL_OUTPUT}")
print("="*70)

Loading model...
Model loaded OK
Video: 1920x1080 @ 25.0 FPS | 7501 frames (~300 sec)
Starting full inference...
requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 2 packages in 420ms
Prepared 1 package in 92ms
Installed 1 package in 9ms
 + lap==0.5.12

requirements: AutoUpdate success ✅ 1.3s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect

Frame     50/7501 | Inst FPS: 16.4 | Avg: 16.3 | ETA ~22.3 min
Frame    100/7501 | Inst FPS: 16.8 | Avg: 16.6 | ETA ~16.4 min
Frame    150/7501 | Inst FPS: 15.8 | Avg: 12.8 | ETA ~15.2 min
Frame    200/7501 | Inst FPS: 16.3 | Avg: 16.3 | ETA ~14.1 min
Frame    250/7501 | Inst FPS: 16.1 | Avg: 16.5 | ETA ~13.3 min
Frame    300/7501 | Inst FPS: 16.4 | Avg: 14.1 | ETA ~13.1 min
Frame    350/7501 | Inst FPS: 16.9 | Avg: 16.5 | ETA ~12.7 min
Frame    400/7501 | Inst FPS: 16.2 | Avg: 16.5 | ETA ~12.3 min
Frame    450/7501 | Inst F